# Generate Output
This notebook will read in the metadata from the [Arquin Spreadsheet](https://docs.google.com/spreadsheets/d/1LRbios7yQRo3aqCh0Es2Wiae_dicg_OtL-_yqP-Tb8I/edit#gid=1718343431) and product an output for ingestion into Omeka 

In [116]:
url = 'https://docs.google.com/spreadsheets/d/1LRbios7yQRo3aqCh0Es2Wiae_dicg_OtL-_yqP-Tb8I/edit#gid=1718343431'

## import libraries

In [117]:
from IPython.display import clear_output
import pandas as pd
from pandas import Series
import numpy as np
import re
import csv
from gsheets import Sheets
from datetime import datetime

In [118]:
%run functions.ipynb

/var/folders/ts/9ppg2hqj4t5_9cslr1528y_m0000gp/T/ipykernel_49992/3168744551.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(file_path
/var/folders/ts/9ppg2hqj4t5_9cslr1528y_m0000gp/T/ipykernel_49992/3168744551.py:2: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(file_path
b'Skipping line 451800: expected 13 fields, saw 14\n'
/var/folders/ts/9ppg2hqj4t5_9cslr1528y_m0000gp/T/ipykernel_49992/3168744551.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path
/var/folders/ts/9ppg2hqj4t5_9cslr1528y_m0000gp/T/ipykernel_49992/3168744551.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = 

## Load the spreadsheet information

In [119]:
# if HttpAccessTokenRefreshError, close everything and start over
sheets = Sheets.from_files('~/client_secrets.json', '~/storage.json')
s = sheets.get(url)

## This is testing to figure out how to get all the data where it needs to go!

### Get the list of Identifiers by Box Number

In [120]:
# get the list of boxes containing metadata

box_list = []
box_list_sheets = s.sheets.titles()

for b in box_list_sheets:
    if 'Box' in b:
        box_list.append(int(b[4:]))

In [6]:
box_list

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69]

In [141]:
x = return_box(box_list[0])

In [142]:
# concepts[concepts['join_concept'].notna()]
'''strip leading and trailing space'''
 
x['title'] = x['title'].str.strip()

### Get the title translations
from Box 'unique_titles' or 'unique_titles_06142022' - pending clarification

In [ ]:
title_translations = s.find('unique_titles').to_frame()

In [ ]:
FIELDS_TO_RETURN = ['identifier', 'media', 'title', 'subject', 'description', 'publisher', 
                 'date', 'rights', 'language', 'relation', 'format', 'type', 'coverage', 'spatial']

In [ ]:
x[FIELDS_TO_RETURN]

In [ ]:
box_with_trans = x.merge(title_translations, how='left', left_on='title', right_on='Title', indicator=True)

In [ ]:
box_with_trans.columns

In [ ]:
FIELDS_PLUS_TITLES = ['Spanish (from Google)', 'Translation ES', 'Portuguese(from Google)', 'Translation PT', '_merge']

In [ ]:
FIELDS_PLUS_TITLES + FIELDS_TO_RETURN

In [ ]:
box_with_trans[FIELDS_TO_RETURN + FIELDS_PLUS_TITLES]

## Load Arquin Metadata with validated Subjects for parsing subjects

In [ ]:
%run arquin_metadata_subjects_review_final.ipynb

In [ ]:
box_with_trans[box_with_trans['identifier']=='A-1 02']

In [ ]:
box_with_trans_and_subjects = box_with_trans.merge(df, on=['media'])

In [ ]:
SUBJECT_TERMS = ['SUBJECT_ID', 'TERM_ID', 'LANGUAGE_CODE', 'use_concept', 'BoxId', 'color']

## Rename the columns post join 

In [ ]:
box_with_trans_and_subjects = box_with_trans_and_subjects.rename(columns={'identifier_x': 'identifier', 
'media': 'media', 
'title_x': 'title', 
'subject_x': 'subject', 
'description_x': 'description', 
'creator_x': 'creator', 
'publisher_x': 'publisher', 
'date_x': 'date', 
'rights_x': 'rights', 
'language_x': 'language', 
'relation_x': 'relation', 
'format_x': 'format', 
'type_x': 'type', 
'coverage_x': 'coverage', 
'spatial_x': 'spatial', 
'Index': 'Index', 
'Title': 'Title', 
'Spanish (from Google)': 'Spanish (from Google)', 
'Translation ES': 'Translation ES', 
'EmptyColumn': 'EmptyColumn', 
'Portuguese(from Google)': 'Portuguese(from Google)', 
'Translation PT': 'Translation PT', 
'_merge': '_merge', 
'identifier_y': 'identifier_y', 
'title_y': 'title_y', 
'subject_y': 'subject_y', 
'description_y': 'description_y', 
'creator_y': 'creator_y', 
'publisher_y': 'publisher_y', 
'date_y': 'date_y', 
'rights_y': 'rights_y', 
'language_y': 'language_y', 
'relation_y': 'relation_y', 
'format_y': 'format_y', 
'type_y': 'type_y', 
'coverage_y': 'coverage_y', 
'spatial_y': 'spatial_y', 
'subjects': 'subjects', 
'join_concept': 'join_concept', 
'extra_notes': 'extra_notes', 
'unnamed: 2': 'unnamed: 2', 
'unnamed: 4': 'unnamed: 4', 
'AACR2_FLAG': 'AACR2_FLAG', 
'DISPLAY_DATE': 'DISPLAY_DATE', 
'DISPLAY_NAME': 'DISPLAY_NAME', 
'DISPLAY_ORDER': 'DISPLAY_ORDER', 
'END_DATE': 'END_DATE', 
'HISTORIC_FLAG': 'HISTORIC_FLAG', 
'OTHER_FLAGS': 'OTHER_FLAGS', 
'PREFERRED_x': 'PREFERRED', 
'START_DATE': 'START_DATE', 
'SUBJECT_ID_x': 'SUBJECT_ID', 
'TERM': 'TERM', 
'TERM_ID': 'TERM_ID', 
'VERNACULAR': 'VERNACULAR', 
'LANGUAGE_CODE': 'LANGUAGE_CODE', 
'PREFERRED_y': 'PREFERRED_y', 
'SUBJECT_ID_y': 'SUBJECT_ID_y', 
'QUALIFIER': 'QUALIFIER', 
'TERM_TYPE': 'TERM_TYPE', 
'PART_OF_SPEECH': 'PART_OF_SPEECH', 
'LANG_STAT': 'LANG_STAT', 
'full_concept': 'full_concept', 
'use_concept': 'use_concept', 
'BoxId': 'BoxId', 
'Unnamed: 0': 'Unnamed: 0', 
'color': 'color', 
'RGB': 'RGB'})

In [ ]:
# show all the columns up to 500
pd.set_option('display.max_columns', 500)

In [ ]:
box_with_trans_and_subjects[FIELDS_TO_RETURN + FIELDS_PLUS_TITLES+ SUBJECT_TERMS]

## Generate a distinct list of subjects for the box

In [ ]:
box_with_trans_and_subjects[['SUBJECT_ID', 'TERM_ID', 'use_concept']].drop_duplicates()

## This will be the script for outputting the file to the CSV

In [ ]:
box_with_trans_and_subjects[output].to_csv(
    path_or_buf='./csv_files/output_file.csv',
    index=False, header=dc_output,
    quoting=csv.QUOTE_NONNUMERIC
)

## Define the Output Columns and labels

In [ ]:
output = ['identifier', 'media', 'title', 'subject','description', 'creator', 'publisher', 'date', 'rights', 'language', 'relation', 'format', 'type', 'coverage', 'spatial']

In [ ]:
dc_output = ['dcterms:identifier',
'dcterms:media',
'dcterms:title',
'dcterms:subject',
'dcterms:description',
'dcterms:creator',
'dcterms:publisher',
'dcterms:date',
'dcterms:rights',
'dcterms:language',
'dcterms:relation',
'dcterms:format',
'dcterms:type',
'dcterms:coverage',
'dcterms:spatial']

## Create the Combination Columns for Title and Translations

In [ ]:
title_cols = ['title', 'Spanish (from Google)', 'Portuguese(from Google)']

In [ ]:
box_with_trans_and_subjects['title_trans'] = box_with_trans_and_subjects[title_cols].apply(lambda title_cols: ', '.join(title_cols.dropna()), axis=1)

## Rename the columns post join 

In [182]:
box_with_trans_and_subjects = box_with_trans_and_subjects.rename(columns={'identifier_x': 'identifier', 
'media': 'media', 
'title_x': 'title', 
'subject_x': 'subject', 
'description_x': 'description', 
'creator_x': 'creator', 
'publisher_x': 'publisher', 
'date_x': 'date', 
'rights_x': 'rights', 
'language_x': 'language', 
'relation_x': 'relation', 
'format_x': 'format', 
'type_x': 'type', 
'coverage_x': 'coverage', 
'spatial_x': 'spatial', 
'Index': 'Index', 
'Title': 'Title', 
'Spanish (from Google)': 'Spanish (from Google)', 
'Translation ES': 'Translation ES', 
'EmptyColumn': 'EmptyColumn', 
'Portuguese(from Google)': 'Portuguese(from Google)', 
'Translation PT': 'Translation PT', 
'_merge': '_merge', 
'identifier_y': 'identifier_y', 
'title_y': 'title_y', 
'subject_y': 'subject_y', 
'description_y': 'description_y', 
'creator_y': 'creator_y', 
'publisher_y': 'publisher_y', 
'date_y': 'date_y', 
'rights_y': 'rights_y', 
'language_y': 'language_y', 
'relation_y': 'relation_y', 
'format_y': 'format_y', 
'type_y': 'type_y', 
'coverage_y': 'coverage_y', 
'spatial_y': 'spatial_y', 
'subjects': 'subjects', 
'join_concept': 'join_concept', 
'extra_notes': 'extra_notes', 
'unnamed: 2': 'unnamed: 2', 
'unnamed: 4': 'unnamed: 4', 
'AACR2_FLAG': 'AACR2_FLAG', 
'DISPLAY_DATE': 'DISPLAY_DATE', 
'DISPLAY_NAME': 'DISPLAY_NAME', 
'DISPLAY_ORDER': 'DISPLAY_ORDER', 
'END_DATE': 'END_DATE', 
'HISTORIC_FLAG': 'HISTORIC_FLAG', 
'OTHER_FLAGS': 'OTHER_FLAGS', 
'PREFERRED_x': 'PREFERRED', 
'START_DATE': 'START_DATE', 
'SUBJECT_ID_x': 'SUBJECT_ID', 
'TERM': 'TERM', 
'TERM_ID': 'TERM_ID', 
'VERNACULAR': 'VERNACULAR', 
'LANGUAGE_CODE': 'LANGUAGE_CODE', 
'PREFERRED_y': 'PREFERRED_y', 
'SUBJECT_ID_y': 'SUBJECT_ID_y', 
'QUALIFIER': 'QUALIFIER', 
'TERM_TYPE': 'TERM_TYPE', 
'PART_OF_SPEECH': 'PART_OF_SPEECH', 
'LANG_STAT': 'LANG_STAT', 
'full_concept': 'full_concept', 
'use_concept': 'use_concept', 
'BoxId': 'BoxId', 
'Unnamed: 0': 'Unnamed: 0', 
'color': 'color', 
'RGB': 'RGB'})

In [190]:
# show all the columns up to 500
pd.set_option('display.max_columns', 500)

In [191]:
box_with_trans_and_subjects[FIELDS_TO_RETURN + FIELDS_PLUS_TITLES+ SUBJECT_TERMS]

,identifier,media,title,subject,description,publisher,date,rights,language,relation,format,type,coverage,spatial,Spanish (from Google),Translation ES,Portuguese(from Google),Translation PT,_merge,SUBJECT_ID,TERM_ID,LANGUAGE_CODE,use_concept,BoxId,color
0,A-1 02,A-1_Arquin_001_0002.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,51.2093/ 3.2247,"Bruges (inhabited place), West Flanders (provi...","Ostend Gate, el Ezelpoort",NaN,"OSTEND GATE, O EZELPOORT",NaN,both,300005101,1000005101,70051,water gates,Box 1,orange
1,A-1 02,A-1_Arquin_001_0002.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,51.2093/ 3.2247,"Bruges (inhabited place), West Flanders (provi...","Ostend Gate, el Ezelpoort",NaN,"OSTEND GATE, O EZELPOORT",NaN,both,300007838,1000007838,70051,arch bridges,Box 1,orange
2,A-1 02,A-1_Arquin_001_0002.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,51.2093/ 3.2247,"Bruges (inhabited place), West Flanders (provi...","Ostend Gate, el Ezelpoort",NaN,"OSTEND GATE, O EZELPOORT",NaN,both,300006075,1000006075,70051,canals (waterways),Box 1,orange
3,A-1 03,A-1_Arquin_001_0003.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,51.2093/ 3.2247,"Bruges (inhabited place), West Flanders (provi...","Ostend Gate, el Ezelpoort",NaN,"OSTEND GATE, O EZELPOORT",NaN,both,300005101,1000005101,70051,water gates,Box 1,orange
4,A-1 03,A-1_Arquin_001_0003.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,51.2093/ 3.2247,"Bruges (inhabited place), West Flanders (provi...","Ostend Gate, el Ezelpoort",NaN,"OSTEND GATE, O EZELPOORT",NaN,both,300007838,1000007838,70051,arch bridges,Box 1,orange
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,A-1 266,A-1_Arquin_001_0266.jpg,Church of Our Lady of Victories at the Sablon,"churches (buildings), architectural sculpture",Notre Dame Du Sablon,Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,"50.8503° N, 4.3517° E","Brussels (inhabited place), Brussels-Capital R...",Iglesia de Nuestra Señora de las Victorias en ...,NaN,NaN,NaN,both,300379328,1000539499,70051,architectural sculpture,Box 1,orange
680,A-1 267,A-1_Arquin_001_0267.jpg,Church of Our Lady of Victories at the Sablon,"churches (buildings), architectural sculpture",Notre Dame Du Sablon,Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,"50.8503° N, 4.3517° E","Brussels (inhabited place), Brussels-Capital R...",Iglesia de Nuestra Señora de las Victorias en ...,NaN,NaN,NaN,both,300007466,1000007466,70051,churches (buildings),Box 1,orange
681,A-1 267,A-1_Arquin_001_0267.jpg,Church of Our Lady of Victories at the Sablon,"churches (buildings), architectural sculpture",Notre Dame Du Sablon,Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,"50.8503° N, 4.3517° E","Brussels (inhabited place), Brussels-Capital R...",Iglesia de Nuestra Señora de las Victorias en ...,NaN,NaN,NaN,both,300379328,1000539499,70051,architectural sculpture,Box 1,orange
682,A-1 268,A-1_Arquin_001_0268.jpg,Church of Our Lady of Victories at the Sablon,"churches (buildings), architectural sculpture",Notre Dame Du Sablon,Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,sti

## Generate a distinct list of subjects for the box

In [199]:
box_with_trans_and_subjects[['SUBJECT_ID', 'TERM_ID', 'use_concept']].drop_duplicates()

,SUBJECT_ID,TERM_ID,use_concept
0,300005101,1000005101,water gates
1,300007838,1000007838,arch bridges
2,300006075,1000006075,canals (waterways)
15,300006273,1000006273,windmills
23,300008247,1000008247,streets
...,...,...,...
545,300076641,1000076641,portals
546,300379328,1000539499,architectural sculpture
563,300180499,1000181542,neon lights
565,300417219,1000772433,brand names


## This will be the script for outputting the file to the CSV

In [222]:
box_with_trans_and_subjects[output].to_csv(
    path_or_buf='./csv_files/output_file.csv',
    index=False, header=dc_output,
    quoting=csv.QUOTE_NONNUMERIC
)

## Define the Output Columns and labels

In [211]:
output = ['identifier', 'media', 'title', 'subject','description', 'creator', 'publisher', 'date', 'rights', 'language', 'relation', 'format', 'type', 'coverage', 'spatial']

In [217]:
dc_output = ['dcterms:identifier',
'dcterms:media',
'dcterms:title',
'dcterms:subject',
'dcterms:description',
'dcterms:creator',
'dcterms:publisher',
'dcterms:date',
'dcterms:rights',
'dcterms:language',
'dcterms:relation',
'dcterms:format',
'dcterms:type',
'dcterms:coverage',
'dcterms:spatial']

## Create the Combination Columns for Title and Translations

In [297]:
title_cols = ['title', 'Spanish (from Google)', 'Portuguese(from Google)']

In [300]:
box_with_trans_and_subjects['title_trans'] = box_with_trans_and_subjects[title_cols].apply(lambda title_cols: ', '.join(title_cols.dropna()), axis=1)